In [2]:
import numpy as np
import pandas as pd

In [3]:
dists = pd.read_csv("klm_raw_inputs/distances.csv", index_col=[0,1])

## Walk Zone

In [82]:
walk = pd.DataFrame(
    np.where(dists.values < 0.5, 1, 0),
    index=dists.index,
    columns=dists.columns
)
walk

413-GE-KG  413-SA-KG  420-GE-KG  420-MS-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            0          0          0          0   
60750101002       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
                  spanish             0          0          0          0   
60750102001       no_match            0          0          0          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
60759806001       no_match            0          0          0          0   
                  spanish             0          0          0          0   
60759809001       no_match            0          0          0          0   

                              420-SE-KG  420-SN-KG  435-GE-KG  449-FB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            0          0          0          0   
60750101002       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
                  spanish             0          0          0          0   
60750102001       no_match            0          0          0          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
60759806001       no_match            0          0          0          0   
                  spanish             0          0          0          0   
60759809001       no_match            0          0          0          0   

                              449-GE-KG  453-GE-KG  ...  859-GE-KG  862-CB-KG  \
census_blockgroup lang_match                        ...                         
60750101001       no_match            0          0  ...          0          0   
60750101002       cantonese           0          0  ...          0          0   
                  no_match            0          0  ...          0          0   
                  spanish             0          0  ...          0          0   
60750102001       no_match            0          0  ...          0          0   
...                                 ...        ...  ...        ...        ...   
60759805011       cantonese           0          0  ...          0          0   
                  no_match            0          0  ...          0          0   
60759806001       no_match            0          0  ...          0          0   
                  spanish             0          0  ...          0          0   
60759809001       no_match            0          0  ...          0          0   

                              862-GE-KG  867-CB-KG  867-GE-KG  872-CB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            0          0          0          0   
60750101002       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
                  spanish             0          0          0          0   
60750102001       no_match            0          0          0          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
60759806001       no_match            0          0          0          0   
                  spanish             0          0          0          0   
60759809001       no_match            0          0          0          0   

         

In [107]:
walk.to_csv("klm_raw_inputs/walk_zone.csv")

## CTIP x Distance

In [84]:
ctip = pd.read_csv("/Users/katherinementzer/Dropbox/SFUSD/Data/block_to_ctip_category.csv")
translator = pd.read_csv("/Users/katherinementzer/Dropbox/SFUSD/Optimization/block_blockgroup_tract.csv")
ctip = ctip.merge(translator, how="left", on="Block")
ctip["ctip"] = np.where(ctip.ctip_category == 1, 1, 0)
ctip = (
    ctip.rename(columns={"BlockGroup": "census_blockgroup"})
    .groupby("census_blockgroup")
    .first()[
        ["ctip"]
    ]
)
ctip = dists[[]].join(ctip, how="left")

In [85]:
sqrt_dist_ctip1 = np.sqrt(dists).multiply(ctip["ctip"], axis="index")
sqrt_dist_ctip0 = np.sqrt(dists).multiply(1-ctip["ctip"], axis="index")

In [108]:
sqrt_dist_ctip0.to_csv("klm_raw_inputs/sqrt_dist_noctip.csv")
sqrt_dist_ctip1.to_csv("klm_raw_inputs/sqrt_dist_ctip.csv")

## Attendance Area

In [20]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
sns.set()

In [87]:
path = os.path.expanduser('~/Downloads/drive-download-20200216T210200Z-001/2013 ESAAs SFUSD.shp')
aasf = gpd.read_file(path).to_crs(epsg=3395)

In [93]:
path = "~/SFUSD/Census 2010_ Blocks for San Francisco/geo_export_d4e9e90c-ff77-4dc9-a766-6a1a7f7d9f9c.shp"
bgsf = gpd.read_file(os.path.expanduser(path))
bgsf["geoid10"] = bgsf["geoid10"].fillna(value=0).astype("int64", copy=False)
translator = pd.read_csv("~/Dropbox/SFUSD/Optimization/block_blockgroup_tract.csv")
translator["Block"] = translator["Block"].fillna(value=0).astype("int64", copy=False)
bgsf = bgsf.merge(translator, how="left", left_on="geoid10", right_on="Block")
bgsf = (
    bgsf.dissolve(by="BlockGroup", as_index=False)
    .to_crs(epsg=3395)
    .rename(columns={"BlockGroup": "census_blockgroup"})
)
bgsf.geometry = bgsf.geometry.centroid
ref = pd.read_csv("klm_raw_inputs/typecodes.csv", index_col=0)[
    ["census_blockgroup", "lang_match"]
]
bgsf = gpd.GeoDataFrame(ref.merge(bgsf, how="left"))

In [94]:
ref = pd.read_csv("klm_raw_inputs/program_idxs.csv", index_col=0)
ref.loc[:, "school_id"] = ref.program_id.apply(lambda x: int(x[:3]))

sc_df = pd.read_csv("~/SFUSD/Data/Cleaned/schools_rehauled_1819.csv")
sc_df = ref.merge(sc_df, how="left", on="school_id")
geometry = [Point(xy) for xy in zip(sc_df['lon'], sc_df['lat'])]
school_gdf = gpd.GeoDataFrame(sc_df, geometry=geometry, crs='epsg:4326').to_crs(epsg=3395)

In [95]:
sch_aa = gpd.sjoin(school_gdf, aasf, how="inner", op='intersects')[["program_id", "index_right"]]

In [96]:
bg_aa = gpd.sjoin(bgsf, aasf, how="inner", op="intersects")[
    ["census_blockgroup", "lang_match", "index_right"]
]

In [98]:
aa_ind = bg_aa.merge(sch_aa)
aa_ind["indicator"] = 1
aa_match = pd.pivot_table(aa_ind, columns="program_id", index=["census_blockgroup", "lang_match"], values="indicator", fill_value=0)

In [109]:
aa_match.to_csv("klm_raw_inputs/aa_match.csv")

## Language Match

In [105]:
cantonese_progs = {"CE","CB", "CN", "CT"}
spanish_progs = {"NS","SB","SE","SN"}
spanish_cols = [x for x in dists.columns if x[4:6] in spanish_progs]
cantonese_cols = [x for x in dists.columns if x[4:6] in cantonese_progs]

In [106]:
lang_match = pd.DataFrame(index=dists.index, columns=dists.columns).fillna(0)
idx = pd.IndexSlice
lang_match.loc[idx[:, "spanish"], spanish_cols] = 1
lang_match.loc[idx[:, "cantonese"], cantonese_cols] = 1
lang_match

413-GE-KG  413-SA-KG  420-GE-KG  420-MS-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            0          0          0          0   
60750101002       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
                  spanish             0          0          0          0   
60750102001       no_match            0          0          0          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
60759806001       no_match            0          0          0          0   
                  spanish             0          0          0          0   
60759809001       no_match            0          0          0          0   

                              420-SE-KG  420-SN-KG  435-GE-KG  449-FB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            0          0          0          0   
60750101002       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
                  spanish             1          1          0          0   
60750102001       no_match            0          0          0          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           0          0          0          0   
                  no_match            0          0          0          0   
60759806001       no_match            0          0          0          0   
                  spanish             1          1          0          0   
60759809001       no_match            0          0          0          0   

                              449-GE-KG  453-GE-KG  ...  859-GE-KG  862-CB-KG  \
census_blockgroup lang_match                        ...                         
60750101001       no_match            0          0  ...          0          0   
60750101002       cantonese           0          0  ...          0          1   
                  no_match            0          0  ...          0          0   
                  spanish             0          0  ...          0          0   
60750102001       no_match            0          0  ...          0          0   
...                                 ...        ...  ...        ...        ...   
60759805011       cantonese           0          0  ...          0          1   
                  no_match            0          0  ...          0          0   
60759806001       no_match            0          0  ...          0          0   
                  spanish             0          0  ...          0          0   
60759809001       no_match            0          0  ...          0          0   

                              862-GE-KG  867-CB-KG  867-GE-KG  872-CB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            0          0          0          0   
60750101002       cantonese           0          1          0          1   
                  no_match            0          0          0          0   
                  spanish             0          0          0          0   
60750102001       no_match            0          0          0          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           0          1          0          1   
                  no_match            0          0          0          0   
60759806001       no_match            0          0          0          0   
                  spanish             0          0          0          0   
60759809001       no_match            0          0          0          0   

         

In [110]:
lang_match.to_csv("klm_raw_inputs/lang_match.csv")

# Backing Out Fixed Effects

In [3]:
estimates = np.load("/Users/katherinementzer/Dropbox/SFUSD/Choice_Model/Oct1estimates.npy")

In [4]:
estimates.shape

(4772, 159)

In [52]:
st = pd.read_csv("~/SFUSD/Data/Cleaned/drop_optout_1819.csv", index_col=0)
st = st.loc[st.grade=="KG"].set_index("studentno").sort_index()

/Users/katherinementzer/opt/anaconda3/envs/geo_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (58) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
st[["sibling", "idschoolattendance", "homelang_desc", "ctip1"]]#.loc[888022999, :]

,sibling,idschoolattendance,homelang_desc,ctip1
studentno,,,,
10210005,NaN,670.0,EN-English,0.0
10210006,[729],521.0,EN-English,0.0
10210007,NaN,735.0,EN-English,0.0
10210010,NaN,505.0,EN-English,0.0
10210039,NaN,820.0,SP-Spanish,0.0
...,...,...,...,...
888022985,[493],481.0,EN-English,0.0
888022990,NaN,820.0,NaN,NaN
888022996,NaN,842.0,CC-Chinese Cantonese,0.0


In [38]:
st[["studentno", "r1_programs", "r2_programs", "r3_programs"]]

,studentno,r1_programs,r2_programs,r3_programs
0,10210005,"['GE', 'GE']",NaN,NaN
1,10210006,"['SE', 'SE', 'SE', 'CE', 'GE']",NaN,NaN
2,10210007,"['JE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE...",NaN,NaN
3,10210010,"['GE', 'SE', 'GE', 'GE', 'GE', 'SE', 'CE', 'GE...",NaN,NaN
4,10210039,"['SN', 'SN', 'SB', 'SN', 'GE', 'SB', 'GE', 'SN']",NaN,NaN
...,...,...,...,...
4767,888021963,NaN,NaN,['GE']
4768,888021967,NaN,NaN,['GE']
4769,888022647,NaN,NaN,['GE']
4770,888022874,NaN,NaN,['SN']


In [41]:
dists = pd.read_csv(
    "/Users/katherinementzer/Documents/sfusd/local_runs/Data/Precomputed/student_program_distances_dropoptout_1819.csv",
    index_col=0,
)

In [42]:
dists

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,4.473374,4.473374,3.384466,3.384466,3.384466,3.384466,3.951158,5.701885,5.701885,5.019194,...,5.513035,1.675126,1.675126,3.880911,3.880911,6.987676,6.987676,1.921483,1.921483,1.921483
10210006,6.380952,6.380952,3.290596,3.290596,3.290596,3.290596,5.772333,4.248004,4.248004,0.824518,...,4.727517,5.484468,5.484468,0.387022,0.387022,5.801425,5.801425,3.844362,3.844362,3.844362
10210007,2.159396,2.159396,1.841366,1.841366,1.841366,1.841366,1.855577,2.009324,2.009324,5.228952,...,1.258508,4.295509,4.295509,4.997587,4.997587,2.515562,2.515562,2.831248,2.831248,2.831248
10210010,2.741758,2.741758,0.415018,0.415018,0.415018,0.415018,2.170503,2.329250,2.329250,4.077301,...,2.041698,3.446032,3.446032,3.638238,3.638238,3.516507,3.516507,1.600115,1.600115,1.600115
10210039,5.082246,5.082246,2.959895,2.959895,2.959895,2.959895,4.482379,5.094352,5.094352,3.582217,...,5.125863,3.009039,3.009039,2.409244,2.409244,6.551905,6.551905,2.142958,2.142958,2.142958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,5.131228,5.131228,2.250961,2.250961,2.250961,2.250961,4.511612,3.928583,3.928583,2.185731,...,4.161934,4.033820,4.033820,1.200044,1.200044,5.471858,5.471858,2.424157,2.424157,2.424157
888022990,4.897837,4.897837,3.083206,3.083206,3.083206,3.083206,4.316363,5.308858,5.308858,4.060572,...,5.267046,2.582724,2.582724,2.894940,2.894940,6.718336,6.718336,2.010533,2.010533,2.010533
888022996,4.630085,4.630085,2.549501,2.549501,2.549501,2.549501,4.027605,4.741816,4.741816,3.628875,...,4.728295,2.725779,2.725779,2.513839,2.513839,6.169357,6.169357,1.687943,1.687943,1.687943


In [46]:
sqrt_dist_ctip1 = np.sqrt(dists).multiply(st["ctip1"], axis="index")
sqrt_dist_ctip0 = np.sqrt(dists).multiply(1-st["ctip1"], axis="index")

In [47]:
sqrt_dist_ctip0

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,2.115035,2.115035,1.839692,1.839692,1.839692,1.839692,1.987752,2.387862,2.387862,2.240356,...,2.347985,1.294267,1.294267,1.970003,1.970003,2.643421,2.643421,1.386176,1.386176,1.386176
10210006,2.526055,2.526055,1.814000,1.814000,1.814000,1.814000,2.402568,2.061069,2.061069,0.908030,...,2.174285,2.341894,2.341894,0.622111,0.622111,2.408615,2.408615,1.960705,1.960705,1.960705
10210007,1.469488,1.469488,1.356969,1.356969,1.356969,1.356969,1.362196,1.417506,1.417506,2.286690,...,1.121832,2.072561,2.072561,2.235528,2.235528,1.586052,1.586052,1.682631,1.682631,1.682631
10210010,1.655825,1.655825,0.644219,0.644219,0.644219,0.644219,1.473263,1.526188,1.526188,2.019233,...,1.428880,1.856349,1.856349,1.907416,1.907416,1.875235,1.875235,1.264957,1.264957,1.264957
10210039,2.254384,2.254384,1.720434,1.720434,1.720434,1.720434,2.117163,2.257067,2.257067,1.892675,...,2.264037,1.734658,1.734658,1.552174,1.552174,2.559669,2.559669,1.463884,1.463884,1.463884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,2.265222,2.265222,1.500320,1.500320,1.500320,1.500320,2.124056,1.982065,1.982065,1.478422,...,2.040082,2.008437,2.008437,1.095465,1.095465,2.339200,2.339200,1.556971,1.556971,1.556971
888022990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
888022996,2.151763,2.151763,1.596716,1.596716,1.596716,1.596716,2.006889,2.177571,2.177571,1.904961,...,2.174464,1.650993,1.650993,1.585509,1.585509,2.483819,2.483819,1.299209,1.299209,1.299209


In [56]:
progs = pd.DataFrame(zip(dists.columns, np.zeros(159)), columns=["programs", "key"])

sib = st[["sibling"]].reset_index()
sib.loc[:, "key"] = 0
sib = sib.merge(progs, on="key")
sib

,studentno,sibling,key,programs
0,10210005,NaN,0,413-GE-KG
1,10210005,NaN,0,413-SA-KG
2,10210005,NaN,0,420-GE-KG
3,10210005,NaN,0,420-MS-KG
4,10210005,NaN,0,420-SE-KG
...,...,...,...,...
758743,888022999,NaN,0,872-CB-KG
758744,888022999,NaN,0,872-GE-KG
758745,888022999,NaN,0,876-CE-KG
758746,888022999,NaN,0,876-CN-KG


In [58]:
sib.loc[:, "indicator"] = 0

sib.loc[~sib.sibling.isna(), "indicator"] = sib.apply(lambda x: not pd.isna(x.sibling) and int(x.programs[:3]) in eval(x.sibling), axis=1)
sib

,studentno,sibling,key,programs,indicator
0,10210005,NaN,0,413-GE-KG,0
1,10210005,NaN,0,413-SA-KG,0
2,10210005,NaN,0,420-GE-KG,0
3,10210005,NaN,0,420-MS-KG,0
4,10210005,NaN,0,420-SE-KG,0
...,...,...,...,...,...
758743,888022999,NaN,0,872-CB-KG,0
758744,888022999,NaN,0,872-GE-KG,0
758745,888022999,NaN,0,876-CE-KG,0
758746,888022999,NaN,0,876-CN-KG,0


In [62]:
sib.indicator = sib.indicator.astype(int)
sib2 = pd.pivot_table(sib, index="studentno", columns="programs", values="indicator")
sib2

programs,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888022990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888022996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
walk = pd.DataFrame(np.where(dists.values < 0.5, 1, 0), index=dists.index, columns=dists.columns)
walk

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
10210007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210010,0,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888022990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888022996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
aa_match = st[["idschoolattendance"]].reset_index()
aa_match.loc[:, "key"] = 0
aa_match = aa_match.merge(progs, on="key")
aa_match.loc[:, "indicator"] = 0

aa_match.loc[:, "indicator"] = aa_match.apply(
    lambda x: int(
        not pd.isna(x.idschoolattendance)
        and int(x.programs[:3]) == x.idschoolattendance
    ),
    axis=1,
)
aa = pd.pivot_table(aa_match, index="studentno", columns="programs", values="indicator")
aa

programs,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10210039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888022990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888022996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
PROGRAM_2_LANGS = {
    "CE": "CC",
    "CB": "CC",
    "CN": "CC",
    "CT": "CC",
    "NS": "SP",
    "SB": "SP",
    "SE": "SP",
    "SN": "SP",
    "JB": "JA",
    "JE": "JA",
    "JN": "JA",
    "KE": "KO",
    "KN": "KO",
    "ME": "CM",
    "MN": "CM",
    "GE": "XX",
    "SA": "XX",
    "MS": "XX",
    "MM": "XX",
    "FB": "XX",
    "DT": "XX",
    "NC": "XX",
    "ED": "XX",
    "AF": "XX",
    "DA": "XX",
}
LANGS_2_PROGRAM = {
    "CC": ["CE", "CB", "CN", "CT"],
    "SP": ["NS", "SB", "SE", "SN"],
    "JA": ["JB", "JE", "JN"],
    "KO": ["KE", "KN"],
    "CM": ["ME", "MN"],
    "XX": ["GE", "SA", "MS", "MM", "FB", "DT", "NC", "ED", "AF", "DA"],
}
st.homelang_desc = st.homelang_desc.fillna("EN")
progs.loc[:, "lang"] = progs.programs.apply(lambda x: PROGRAM_2_LANGS[x[4:6]])

In [71]:
lang = st[["homelang_desc"]].reset_index()
lang.loc[:, "key"] = 0
lang = lang.merge(progs, on="key")

lang.loc[:, "indicator"] = lang.apply(
    lambda x: int(x.programs[4:6] in LANGS_2_PROGRAM[x.homelang_desc[:2]]) if 
        x.homelang_desc[:2] in LANGS_2_PROGRAM
        else int(x.programs[4:6] in LANGS_2_PROGRAM["XX"]),
    axis=1,
)

In [72]:
lang_match = pd.pivot_table(lang, index="studentno", columns="programs", values="indicator")
lang_match

programs,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,1,1,1,1,0,0,1,1,1,1,...,1,0,1,0,1,0,1,0,0,1
10210006,1,1,1,1,0,0,1,1,1,1,...,1,0,1,0,1,0,1,0,0,1
10210007,1,1,1,1,0,0,1,1,1,1,...,1,0,1,0,1,0,1,0,0,1
10210010,1,1,1,1,0,0,1,1,1,1,...,1,0,1,0,1,0,1,0,0,1
10210039,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,1,1,1,1,0,0,1,1,1,1,...,1,0,1,0,1,0,1,0,0,1
888022990,1,1,1,1,0,0,1,1,1,1,...,1,0,1,0,1,0,1,0,0,1
888022996,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,1,0


In [77]:
base_utility = np.zeros((4772, 159))
base_utility += sqrt_dist_ctip_param * sqrt_dist_ctip1.values 
base_utility += sqrt_dist_noctip_param * sqrt_dist_ctip0.values 
base_utility += sibling_param * sib2.values 
base_utility += walk_zone_param * walk.values 
base_utility += lang_match_param * lang_match.values 
base_utility += aa_match_param * aa.values 

In [78]:
reconstructed = pd.DataFrame(base_utility, columns=dists.columns, index=dists.index)
reconstructed

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,-4.642099,-4.642099,-3.871137,-3.871137,-5.151137,-5.151137,-4.285705,-5.406013,-5.406013,-4.992996,...,-5.294359,-3.623946,-2.343946,-5.516008,-4.236008,-7.401579,-6.121579,-3.881292,-3.881292,-2.601292
10210006,-5.792953,-5.792953,-3.799200,-3.799200,-5.079200,-5.079200,-5.447190,-4.490993,-4.490993,-1.262484,...,-4.807999,-6.557304,-5.277304,-1.706911,-0.426911,-6.744121,-5.464121,-5.489973,-5.489973,-4.209973
10210007,-2.834567,-2.834567,-2.519514,-2.519514,-3.799514,-3.799514,-2.534148,-2.689017,-2.689017,-5.122732,...,-1.861131,-5.803171,-4.523171,-6.259479,-4.979479,-4.440947,-3.160947,-4.711368,-4.711368,-3.431368
10210010,-3.356311,-3.356311,-0.488813,-0.488813,-1.768813,-1.768813,-2.845136,-2.993327,-2.993327,-4.373852,...,-2.720864,-5.197778,-3.917778,-5.340766,-4.060766,-5.250659,-3.970659,-3.541878,-3.541878,-2.261878
10210039,-6.312274,-6.312274,-4.817216,-4.817216,-3.537216,-3.537216,-5.928056,-6.319788,-6.319788,-5.299489,...,-6.339303,-4.857043,-4.857043,-4.346087,-4.346087,-7.167073,-7.167073,-4.098876,-4.098876,-4.098876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,-5.062620,-5.062620,-2.920897,-2.920897,-4.200897,-4.200897,-4.667356,-4.269783,-4.269783,-2.859581,...,-4.432229,-5.623624,-4.343624,-3.067303,-1.787303,-6.549761,-5.269761,-4.359518,-4.359518,-3.079518
888022990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
888022996,-6.024937,-6.024937,-4.470804,-4.470804,-4.470804,-4.470804,-5.619291,-6.097199,-6.097199,-5.333889,...,-6.088500,-3.342781,-4.622781,-3.159425,-4.439425,-5.674693,-6.954693,-2.357785,-2.357785,-3.637785


In [80]:
diff = pd.DataFrame(
    estimates - reconstructed.values,
    index=reconstructed.index,
    columns=reconstructed.columns,
)
diff

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,-1.682253,-7.169463,-2.620758,-7.988519,-inf,-inf,-1.734954,-4.076179,-inf,-6.146013,...,-3.423013,-inf,-inf,-inf,-inf,2.864858,-3.234034,-10.961531,-inf,-inf
10210006,-1.686018,-7.173228,-2.620523,-7.988283,-0.965587,-1.402487,-1.738753,-3.353610,-3.100081,-8.893086,...,-2.352213,-0.227973,-1.944872,-4.753868,-6.292872,0.593457,-3.507030,-8.968387,-1.159277,-1.783582
10210007,-1.676340,-7.163549,-2.616336,-7.984097,-0.961401,-1.398300,-1.729224,-5.421180,-5.167651,-4.538384,...,-5.061977,-0.745002,-2.461901,0.567213,-0.971790,-0.928566,-2.941583,-6.878371,0.127523,-0.496781
10210010,-1.678047,-7.165256,-2.632169,-7.999930,-0.977233,-1.414133,-1.730241,-4.464318,-4.210788,-4.271050,...,-3.917816,-1.065966,-2.782866,0.821815,-0.717188,1.054462,-3.201777,-9.117767,-0.405176,-1.029480
10210039,-0.403530,-5.890739,-1.339666,-6.707426,-1.059230,-1.496129,-0.456139,-2.519892,-1.080862,-5.293230,...,-1.815766,-1.737591,-2.174490,-inf,-inf,2.412055,-2.039948,-10.595353,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888022985,-2.807406,-8.294615,-3.769127,-9.136888,-0.928691,-1.365591,-2.920423,-3.071272,-1.632243,-7.233927,...,-2.073058,0.678772,-1.038127,-inf,-inf,-0.279313,-3.553176,-9.950629,-inf,-inf
888022990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
888022996,0.360616,-5.126593,-1.410085,-6.777845,-inf,-inf,0.068148,-1.812985,-inf,-4.505052,...,-0.260749,-inf,-inf,-inf,-inf,-0.613700,2.173455,-7.910663,-inf,-inf


In [81]:
st.head()

,r1_ranked_idschool,r1_listed_ranks,r1_programs,grade,r1_randomnumber,randomnumber,requestprogramdesignation,latitude,longitude,r2_ranked_idschool,...,Academic Score,N'hood SES Score,FRL Score,sibling,currentlpsibling,currentlp,msf,aaprek,aa,median_hh_income
studentno,,,,,,,,,,,,,,,,,,,,,
10210005,"[435, 670]","[1, 2]","['GE', 'GE']",KG,"[0.911005150211793, 0.111640986769763]",0.681393,0.0,37.718266,-122.480908,NaN,...,0.365385,0.766113,0.584975,NaN,NaN,NaN,NaN,NaN,[670],96125.0
10210006,"[729, 537, 680, 876, 729]","[1, 2, 3, 4, 5]","['SE', 'SE', 'SE', 'CE', 'GE']",KG,"[0.411758992402522, 0.331974906582103, 0.06240...",0.694158,0.0,37.714647,-122.403722,NaN,...,0.380597,0.640319,0.660631,[729],NaN,NaN,NaN,NaN,NaN,88594.0
10210007,"[786, 569, 735, 435, 549, 413, 589, 479, 718, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['JE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE...",KG,"[0.924068668432651, 0.591082980668961, 0.31875...",0.289696,0.0,37.780102,-122.442890,NaN,...,0.022388,0.504279,0.263899,NaN,NaN,NaN,NaN,NaN,[735],110735.0
10210010,"[796, 420, 505, 722, 589, 618, 876, 718, 493, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","['GE', 'SE', 'GE', 'GE', 'GE', 'SE', 'CE', 'GE...",KG,"[0.0774816984940387, 0.826669827283934, 0.8127...",0.200155,0.0,37.758470,-122.442804,NaN,...,0.000000,0.288537,0.000000,NaN,NaN,NaN,NaN,NaN,[505],146719.0
10210039,"[537, 680, 691, 729, 493, 593, 820, 420]","[1, 2, 3, 4, 5, 6, 7, 8]","['SN', 'SN', 'SB', 'SN', 'GE', 'SB', 'GE', 'SN']",KG,"[0.69786295287675, 0.261372761008729, 0.299331...",0.038574,0.0,37.712834,-122.454351,NaN,...,0.519718,0.728821,0.685261,NaN,NaN,NaN,NaN,NaN,[820],66154.0


In [7]:
fts = pd.read_csv("/Users/katherinementzer/code/choice-model/local_outputs/computed_data/features_1819.csv")

In [76]:
sqrt_dist_ctip_param = -3.0
sqrt_dist_noctip_param = -2.8
walk_zone_param = 0.035
sibling_param =6.44
aa_match_param = 0.93
lang_match_param = 1.28
params = [
    sqrt_dist_ctip_param,
    sqrt_dist_noctip_param,
    walk_zone_param,
    sibling_param,
    aa_match_param,
    lang_match_param,
]
cols = [
    "sqrt_distance_x_ctip1",
    "sqrt_distance_x_ctip0",
    "distance_indicator_lt_0_5",
    "sibling",
    "attn_match",
    "lang_match",
]

In [16]:
fts["base_util"] = fts.apply(lambda x: sum([x[i] * j for i, j in zip(cols, params)]), axis=1)

In [34]:
fts[["studentno", "program_id", "base_util", "lang_match"]].head(10)

,studentno,program_id,base_util,lang_match
0,1819-10210005,413-GE-KG,-4.986686,0
1,1819-10210005,413-SA-KG,-4.986686,0
2,1819-10210005,420-GE-KG,-4.221588,0
3,1819-10210005,420-MS-KG,-4.221588,0
4,1819-10210005,420-SE-KG,-2.941588,1
5,1819-10210005,420-SN-KG,-2.941588,1
6,1819-10210005,435-GE-KG,-4.630660,0
7,1819-10210005,456-GE-KG,-5.182918,0
8,1819-10210005,456-SB-KG,-3.902918,1
9,1819-10210005,618-MM-KG,-5.669326,0


In [39]:
fts.head()

,studentno,program_id,school_413,school_420,school_435,school_449,school_497,school_453,school_456,school_476,...,sqrt_distance,ctip1,ctip0,sqrt_distance_x_ctip1,sqrt_distance_x_ctip0,distance_indicator_lt_0_5,sibling,attn_match,lang_match,base_util
0,1819-10210005,413-GE-KG,1,0,0,0,0,0,0,0,...,2.113102,0,1,0.0,2.113102,0,0,1,0,-4.986686
1,1819-10210005,413-SA-KG,1,0,0,0,0,0,0,0,...,2.113102,0,1,0.0,2.113102,0,0,1,0,-4.986686
2,1819-10210005,420-GE-KG,0,1,0,0,0,0,0,0,...,1.839853,0,1,0.0,1.839853,0,0,1,0,-4.221588
3,1819-10210005,420-MS-KG,0,1,0,0,0,0,0,0,...,1.839853,0,1,0.0,1.839853,0,0,1,0,-4.221588
4,1819-10210005,420-SE-KG,0,1,0,0,0,0,0,0,...,1.839853,0,1,0.0,1.839853,0,0,1,1,-2.941588


In [26]:
reconstructed = pd.pivot_table(fts, columns="program_id", index="studentno", values="base_util")
reconstructed = fts.groupby("studentno").first()[[]].join(reconstructed, how="left")

In [27]:
diff = pd.DataFrame(
    estimates - reconstructed.values,
    index=reconstructed.index,
    columns=reconstructed.columns,
)

In [29]:
diff.head(10)

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
1819-10210005,-1.337666,-6.824875,-2.270307,-7.638068,-inf,-inf,-1.389999,-2.795129,-inf,-5.788568,...,-3.073972,-inf,-inf,-inf,-inf,0.653844,-2.885048,-13.173280,-inf,-inf
1819-10210006,-1.335752,-6.822961,-2.271673,-7.639434,-3.176738,-3.613637,-1.388217,-2.078876,-1.825346,-8.540377,...,-2.007330,-2.431307,-1.588206,-6.962326,-5.941329,-1.622708,-3.163194,-11.174822,-3.365712,-1.430017
1819-10210007,-1.321523,-6.808732,-2.269649,-7.637409,-3.174713,-3.611613,-1.374941,-4.136620,-3.883090,-4.189976,...,-4.708304,-2.953882,-2.110781,-1.646836,-0.625840,-3.135690,-2.588708,-9.090968,-2.085074,-0.149378
1819-10210010,-1.326253,-6.813462,-2.282399,-7.650160,-3.187464,-3.624363,-1.377832,-3.181687,-2.928158,-3.920053,...,-3.568320,-3.271758,-2.428657,-1.390389,-0.369392,-1.156150,-2.852390,-11.326956,-2.614365,-0.678670
1819-10210039,-1.338106,-6.825315,-2.273171,-7.640932,-1.992735,-2.429635,-1.390667,-2.522160,-1.083130,-6.217031,...,-2.749769,-3.945040,-3.101940,-inf,-inf,0.198249,-2.973754,-12.808543,-inf,-inf
1819-10210053,-1.331017,-6.818226,-2.264595,-7.632356,-inf,-inf,-1.383193,-1.737886,-inf,-5.268939,...,-2.056780,-inf,-inf,-inf,-inf,-1.699547,-3.120804,-11.050886,-inf,-inf
1819-10210055,-1.331017,-6.818226,-2.264595,-7.632356,-inf,-inf,-1.383193,-1.737886,-inf,-5.268939,...,-2.056780,-inf,-inf,-inf,-inf,-1.699547,-3.120804,-11.050886,-inf,-inf
1819-10210058,-1.320484,-6.807693,-2.270309,-7.638069,-inf,-inf,-1.374923,-4.077997,-inf,-4.328058,...,-4.594423,-inf,-inf,-1.467165,-0.446168,-9.383559,-8.951053,-9.206810,-1.935945,-0.000249
1819-10210066,-1.332872,-6.820082,-2.264080,-7.631841,-3.169145,-3.606044,-1.385022,-2.919825,-2.666296,-5.073133,...,-3.284994,-6.111531,-5.268431,1.001411,2.022408,2.446795,-2.839467,-14.537330,-4.395132,-2.459436


In [30]:
reconstructed.head()

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
1819-10210005,-4.986686,-4.986686,-4.221588,-4.221588,-2.941588,-2.941588,-4.630660,-6.687064,-6.687064,-5.350441,...,-5.643400,-1.413560,-2.693560,-3.312434,-4.592434,-5.190565,-6.470565,-1.669542,-1.669542,-2.949542
1819-10210006,-6.143220,-6.143220,-4.148050,-4.148050,-2.868050,-2.868050,-5.797727,-5.765727,-5.765727,-1.615193,...,-5.152882,-4.353970,-5.633970,0.501546,-0.778454,-4.527957,-5.807957,-3.283538,-3.283538,-4.563538
1819-10210007,-3.189385,-3.189385,-2.866202,-2.866202,-1.586202,-1.586202,-2.888430,-3.973577,-3.973577,-5.471140,...,-2.214804,-3.594291,-4.874291,-4.045430,-5.325430,-2.233822,-3.513822,-2.498771,-2.498771,-3.778771
1819-10210010,-3.708105,-3.708105,-0.838582,-0.838582,0.441418,0.441418,-3.197544,-4.275957,-4.275957,-4.724848,...,-3.070360,-2.991986,-4.271986,-3.128562,-4.408562,-3.040046,-4.320046,-1.332689,-1.332689,-2.612689
1819-10210039,-5.377698,-5.377698,-3.883711,-3.883711,-2.603711,-2.603711,-4.993528,-6.317521,-6.317521,-4.375689,...,-5.405301,-2.649594,-3.929594,-2.141245,-3.421245,-4.953267,-6.233267,-1.885686,-1.885686,-3.165686


In [32]:
 pd.DataFrame(
    estimates,
    index=reconstructed.index,
    columns=reconstructed.columns,
).head()

,413-GE-KG,413-SA-KG,420-GE-KG,420-MS-KG,420-SE-KG,420-SN-KG,435-GE-KG,449-FB-KG,449-GE-KG,453-GE-KG,...,859-GE-KG,862-CB-KG,862-GE-KG,867-CB-KG,867-GE-KG,872-CB-KG,872-GE-KG,876-CE-KG,876-CN-KG,876-GE-KG
studentno,,,,,,,,,,,,,,,,,,,,,
1819-10210005,-6.324352,-11.811562,-6.491895,-11.859656,-inf,-inf,-6.020659,-9.482193,-inf,-11.139009,...,-8.717372,-inf,-inf,-inf,-inf,-4.536721,-9.355613,-14.842823,-inf,-inf
1819-10210006,-7.478972,-12.966181,-6.419723,-11.787484,-6.044788,-6.481687,-7.185944,-7.844603,-7.591073,-10.155570,...,-7.160212,-6.785276,-7.222176,-6.460780,-6.719783,-6.150664,-8.971151,-14.458360,-6.649250,-5.993554
1819-10210007,-4.510908,-9.998117,-5.135851,-10.503611,-4.760915,-5.197815,-4.263371,-8.110197,-7.856668,-9.661117,...,-6.923108,-6.548172,-6.985072,-5.692266,-5.951270,-5.369512,-6.102530,-11.589739,-4.583845,-3.928149
1819-10210010,-5.034358,-10.521567,-3.120982,-8.488743,-2.746046,-3.182946,-4.575377,-7.457645,-7.204115,-8.644902,...,-6.638679,-6.263744,-6.700643,-4.518951,-4.777954,-4.196197,-7.172436,-12.659645,-3.947055,-3.291359
1819-10210039,-6.715804,-12.203013,-6.156882,-11.524643,-4.596446,-5.033346,-6.384195,-8.839680,-7.400651,-10.592719,...,-8.155070,-6.594634,-7.031533,-inf,-inf,-4.755017,-9.207021,-14.694230,-inf,-inf


## Fixed Effects

In [4]:
fake_fixed_effects = pd.DataFrame(
    np.zeros(dists.values.shape), columns=dists.columns, index=dists.index
)
fake_fixed_effects.to_csv("klm_raw_inputs/fixed_effects.csv")

In [9]:
fixed_effects = pd.read_csv("/Users/katherinementzer/SFUSD/oct1_estimates_fixed_effects.csv")
fixed_effects.loc[:, "school_id"] = fixed_effects.school_id.apply(lambda x: int(x[-3:]))
fixed_effects

,school_id,fixed_effect
0,413,-1.152918
1,420,-1.695959
2,435,-0.640214
3,456,-3.486544
4,618,-2.782760
...,...,...
67,876,-1.258745
68,801,-1.975114
69,476,0.000000
70,724,0.000000


In [10]:
df = pd.DataFrame(dists.columns, columns=["program_id"])
df.loc[:,"school_id"] = df.program_id.apply(lambda x: int(x[:3]))
df = df.merge(fixed_effects, how="left", on="school_id")
df

,program_id,school_id,fixed_effect
0,413-GE-KG,413,-1.152918
1,413-SA-KG,413,-1.152918
2,420-GE-KG,420,-1.695959
3,420-MS-KG,420,-1.695959
4,420-SE-KG,420,-1.695959
...,...,...,...
154,872-CB-KG,872,-2.036270
155,872-GE-KG,872,-2.036270
156,876-CE-KG,876,-1.258745
157,876-CN-KG,876,-1.258745


In [13]:
ref = pd.read_csv("klm_raw_inputs/program_idxs.csv")
df = ref.merge(df[["program_id", "fixed_effect"]], how="left", on="program_id")
df

,program_idx,program_id,fixed_effect
0,0,413-GE-KG,-1.152918
1,1,413-SA-KG,-1.152918
2,2,420-GE-KG,-1.695959
3,3,420-MS-KG,-1.695959
4,4,420-SE-KG,-1.695959
...,...,...,...
154,154,872-CB-KG,-2.036270
155,155,872-GE-KG,-2.036270
156,156,876-CE-KG,-1.258745
157,157,876-CN-KG,-1.258745


In [14]:
df.to_csv("klm_raw_inputs/fixed_effects.csv", index=False)

## Program Eligibility Adjustments 

In [16]:
eligibility = pd.DataFrame(
    np.zeros(dists.values.shape), columns=dists.columns, index=dists.index
)

In [24]:
tmp = eligibility[[]].reset_index()
tmp = pd.get_dummies(tmp, columns=["lang_match"], prefix="", prefix_sep="")

In [25]:
tmp

,census_blockgroup,cantonese,no_match,spanish
0,60750101001,0,1,0
1,60750101002,1,0,0
2,60750101002,0,1,0
3,60750101002,0,0,1
4,60750102001,0,1,0
...,...,...,...,...
926,60759805011,1,0,0
927,60759805011,0,1,0
928,60759806001,0,1,0
929,60759806001,0,0,1


In [28]:
spanish = {"SN", "SB"}
cantonese = {"CN", "CT", "CB"}
for col in eligibility.columns:
    if col[4:6] in spanish:
        eligibility.loc[:, col] = tmp.spanish.to_numpy()
    elif col[4:6] in cantonese:
        eligibility.loc[:, col] = tmp.cantonese.to_numpy()
    else:
        eligibility.loc[:, col] = np.ones(len(eligibility), dtype=int)
        

In [29]:
eligibility

413-GE-KG  413-SA-KG  420-GE-KG  420-MS-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            1          1          1          1   
60750101002       cantonese           1          1          1          1   
                  no_match            1          1          1          1   
                  spanish             1          1          1          1   
60750102001       no_match            1          1          1          1   
...                                 ...        ...        ...        ...   
60759805011       cantonese           1          1          1          1   
                  no_match            1          1          1          1   
60759806001       no_match            1          1          1          1   
                  spanish             1          1          1          1   
60759809001       no_match            1          1          1          1   

                              420-SE-KG  420-SN-KG  435-GE-KG  449-FB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            1          0          1          1   
60750101002       cantonese           1          0          1          1   
                  no_match            1          0          1          1   
                  spanish             1          1          1          1   
60750102001       no_match            1          0          1          1   
...                                 ...        ...        ...        ...   
60759805011       cantonese           1          0          1          1   
                  no_match            1          0          1          1   
60759806001       no_match            1          0          1          1   
                  spanish             1          1          1          1   
60759809001       no_match            1          0          1          1   

                              449-GE-KG  453-GE-KG  ...  859-GE-KG  862-CB-KG  \
census_blockgroup lang_match                        ...                         
60750101001       no_match            1          1  ...          1          0   
60750101002       cantonese           1          1  ...          1          1   
                  no_match            1          1  ...          1          0   
                  spanish             1          1  ...          1          0   
60750102001       no_match            1          1  ...          1          0   
...                                 ...        ...  ...        ...        ...   
60759805011       cantonese           1          1  ...          1          1   
                  no_match            1          1  ...          1          0   
60759806001       no_match            1          1  ...          1          0   
                  spanish             1          1  ...          1          0   
60759809001       no_match            1          1  ...          1          0   

                              862-GE-KG  867-CB-KG  867-GE-KG  872-CB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match            1          0          1          0   
60750101002       cantonese           1          1          1          1   
                  no_match            1          0          1          0   
                  spanish             1          0          1          0   
60750102001       no_match            1          0          1          0   
...                                 ...        ...        ...        ...   
60759805011       cantonese           1          1          1          1   
                  no_match            1          0          1          0   
60759806001       no_match            1          0          1          0   
                  spanish             1          0          1          0   
60759809001       no_match            1          0          1          0   

         

In [30]:
eligibility.to_csv("klm_raw_inputs/program_eligibility_adjustment.csv")